# Imports

In [2]:
import os
import datetime
import pickle
import numpy as np
import pandas as pd
# import dask.dataframe as dd
# import vaex
# import modin.pandas as md
import qgrid
qgrid.enable()
# qgrid.disable()

%load_ext autoreload
%autoreload 2
import polygon_rest_api as api

# import ipdb
# ipdb.post_mortem()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from bokeh.io import show
from bokeh.models import ColumnDataSource
from bokeh.palettes import RdBu3
from bokeh.plotting import figure

# Params

In [4]:
symbol='GLD'
fmat = 'parquet'
start_date = '2019-01-01'
end_date = datetime.date.today().isoformat()

# Download Tick data

In [ ]:
# ts = api.get_trades_date(symbol, date='2019-12-03')

ts = api.get_trades_dates(symbol, start_date, end_date)

# Read Tick data

In [5]:
date = '2019-05-08'
ts = api.read_matching_files(f"data/ticks/{fmat}/{symbol}/{symbol}_{date}.{fmat}", format=fmat)

# ts_dd = dd.read_parquet(f"data/ticks/{fmat}/{symbol}/{symbol}_2020-*.{fmat}")

# Process Trades Data

In [6]:
ts['timestamp'] = pd.to_datetime(ts['epoch'], utc=True, unit='ns')

ts['vwap1'] = api.rolling_decay_vwap(ts, window_len=10, decay='none')
ts['vwap2'] = api.rolling_decay_vwap(ts, window_len=10, decay='linear')


In [8]:

p = figure(title=f"{symbol} | {date}", x_axis_type='datetime', plot_width=1300, plot_height=750, output_backend="webgl")

p.multi_line(xs=[ts.timestamp, ts.timestamp, ts.timestamp], ys=[ts.price, ts.vwap1, ts.vwap2], color=["firebrick", "navy", "green"], alpha=[0.2, 0.5, 0.5], line_width=3)

# p.line(x=ts2.timestamp, y=ts2.vwap, line_width=2)
# p.scatter(x=ts2.timestamp, y=ts2.price line_width=2, alpha=0.5)

show(p)


In [ ]:
nrows = list(range(len(ts)))
window_len = 7
vwap = []
for nrow in nrows:
    if nrow >= window_len:
        price = ts['price'][nrow - window_len:nrow].values
        volume = ts['volume'][nrow - window_len:nrow].values
        tmp = 
        vwap.append(tmp)
    else:
        vwap.append(None)
vwap


In [ ]:

c1 = RdBu3[2] # red
c2 = RdBu3[0] # blue

outliers_idx = api.get_outliers(ts, 7)
out = np.where(outliers_idx, c1, c2)
label = np.where(outliers_idx, 'outlier', 'normal')

source = ColumnDataSource(dict(
    x=ts.timestamp,
    y=ts.price,
    color=out,
    label=label
))

p = figure(title=f"{symbol} ticks", x_axis_type='datetime', plot_width=1200, plot_height=700)

# p.line(x='x', y='y', legend="label", line_width=2, source=source)
# p.circle( x='x', y='y', radius=0.5, color='color', legend_group='label', source=source)
p.scatter(x='x', y='y', color='color', legend_group='label', source=source)

show(p)

In [ ]:
# vf = vaex.from_csv(f"data/ticks/csv/{symbol}/{symbol}_2020-01-03.csv")
vf = vaex.from_csv(f"data/ticks/csv/{symbol}/{symbol}_2020-01-03.csv")

# vf = vaex.open('data/ticks/csv/LYFT/*.csv')
vf = vaex.open('data/ticks/parquet/LYFT/*.parquet')

In [ ]:
# ts = df_pd
# ts
# len(ts)
# len(ts[ts.duplicated()]) #/ len(ts)
# len(ts[ts.duplicated(subset='sip_epoch')])
# len(ts[ts.duplicated(subset='timestamp')]) / len(ts)
len(ts[ts.duplicated(subset=['sequence', 'price', 'volume'])]) / len(ts)

In [ ]:
# ts[ts.duplicated(keep=False)].iloc[0:30]
# ts[ts.duplicated(keep=False, subset=['sequence', 'price', 'volume'])].sort_values(['sequence', 'price', 'volume']).iloc[0:30]
# ts[ts.duplicated(keep=False, subset='timestamp')].sort_values('timestamp').iloc[1:30]